In [1]:
# 데이터 불러오기
from dotenv import load_dotenv
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine


In [4]:
# .env 파일에서 환경 부르기
load_dotenv()

# 환경 변수 사용
database = os.getenv("CON_STR")


# # csv 파일 경로
# csv_file_path = os.getenv("CSV_FILE_PATH")
# print(csv_file_path)

engine = create_engine(database)

In [10]:
# -- Arbor_sourcing_M1
# 직전 3개월 데이터 구매 이력 소싱 - raw data
sql = ''' 
select 
	t.YYYYMM, t.id, t.gender, t.age, 
    CASE 
	    WHEN SUM(CASE WHEN b.cigatype = 'CC' THEN 1 ELSE 0 END) > 0 
	 AND SUM(CASE WHEN b.cigatype = 'HnB' THEN 1 ELSE 0 END) > 0 
	THEN 'Mixed' 
	ELSE MAX(b.cigatype)  -- CC 또는 HnB가 없을 경우 가장 큰 값을 사용
	END AS cigatype,
	b.company,
    b.ProductFamilyCode,
    b.productSubFamilyCode,
	b.New_FLAVORSEG,
    b.New_TARSEGMENTAT,
	b.engname,
    sum(a.pack_qty) qty
from  cx.agg_TEREA_Sourcing t
	join cx.fct_K7_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
join cx.product_master_temp b on a.product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  cigatype != 'CSV' AND 4 < LEN(a.id)
where t.YYYYMM='202211'	
group by 
	t.YYYYMM, t.id, t.gender, t.age,
	b.company,
    b.ProductFamilyCode,
    b.productSubFamilyCode,
	b.New_FLAVORSEG,
    b.New_TARSEGMENTAT,
	b.engname
'''

data = pd.read_sql_query(sql=sql, con=engine)


In [11]:
data

,YYYYMM,id,gender,age,cigatype,company,ProductFamilyCode,productSubFamilyCode,New_FLAVORSEG,New_TARSEGMENTAT,engname,qty
0,202211,003DE9124299CC396950FEF717EFE5020EFCB079E8DECB...,남,40대,CC,BAT,DUNHILL,DUNHILLKS,Regular,ULT,DUNHILL 3MG,1.0
1,202211,003DE9124299CC396950FEF717EFE5020EFCB079E8DECB...,남,40대,HnB,BAT,GLO,NEOSTICKS,Regular,None,NEOSTIKS BRIGHT TOBACCO,31.0
2,202211,007BE5AEC24991C378435D650C80E7C168971700CB8EDF...,남,40대,HnB,PMK,IQOS,HEETS,Fresh,None,HEETS BLUE LABEL,3.0
3,202211,007BE5AEC24991C378435D650C80E7C168971700CB8EDF...,남,40대,HnB,PMK,IQOS,HEETS,New Taste,None,HEETS SUMMER BREEZE,1.0
4,202211,00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDE...,남,50대,HnB,PMK,IQOS,HEETS,New Taste,None,HEETS BLACK PURPLE SELECTION,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3295,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,HnB,KTG,LIL,MIIX,New Taste,None,MIIX ICE DOUBLE,2.0
3296,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,KTG,RAISON,RAISON,Regular,ULT,RAISON BLUE,1.0
3297,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,KTG,RAISON,RASIONFR,New Taste,1MG,RAISON FRENCH YOGO,1.0
3298,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,KTG,THEONE,THEONE,Regular,1MG,THE ONE BLUE,1.0


In [14]:
# -- Arbor_sourcing_M1 작업
result = data.groupby(['YYYYMM', 'id', 'gender', 'age']).agg({
    'company': lambda x: x.unique(),
    'New_FLAVORSEG': lambda x: x.unique()
}).reset_index()

result

,YYYYMM,id,gender,age,company,New_FLAVORSEG
0,202211,003DE9124299CC396950FEF717EFE5020EFCB079E8DECB...,남,40대,[BAT ],[Regular]
1,202211,007BE5AEC24991C378435D650C80E7C168971700CB8EDF...,남,40대,[PMK ],"[Fresh, New Taste]"
2,202211,00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDE...,남,50대,[PMK ],"[New Taste, Regular]"
3,202211,00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B3...,남,30대,[KTG ],[New Taste]
4,202211,01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CF...,남,30대,"[KTG , PMK ]","[New Taste, Fresh]"
...,...,...,...,...,...,...
1413,202211,FF83E54787EB82C6293ADA53E369F68215B39162BD33E3...,남,50대,[PMK ],"[Fresh, Regular]"
1414,202211,FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4...,남,20대,[KTG ],[Fresh]
1415,202211,FFC7C4D03CF065384860A5176E832123A07E70B6E18539...,여,50대,[KTG ],[Fresh]
1416,202211,FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4E...,남,40대,[KTG ],[Regular]


In [119]:
 
data['company'] = data['company'].apply(lambda x: ','.join(dict.fromkeys(map(str.strip, x.split(',')))))

data['FLAVOR'] = data['FLAVOR'].apply(lambda x: ','.join(dict.fromkeys(map(str.strip, x.split(',')))))

# data2.to_csv('prac.csv', index=False)

In [9]:
pivot_table = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['company'],
    values=['qty'],
    aggfunc='sum'            
)

pivot_table 

qty  \
company                                                              BAT            
YYYYMM id                                                 gender age                
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대         32.0   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대          NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대          NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대          NaN   
...                                                                           ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대          NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대          NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대          NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대          NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대          1.0   

                                                                                   \
company                                                              JTI            
YYYYMM id                                                 gender age                
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대          NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대          NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대          NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대          NaN   
...                                                                           ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대          NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대          NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대          NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대          NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대          NaN   

                                                                                   \
company                                                              KTG            
YYYYMM id                                                 gender age                
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대          NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대          NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대          9.0   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대          2.0   
...                                                                           ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대          NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대          1.0   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대          4.0   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대          6.0   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대          9.0   

                                                                                   
company                                                              PMK           
YYYYMM id                                                 gender age               
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대          NaN  
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          4.0  
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대         14.0  
 

In [161]:
data['CC_taste'] = data['cigatype'] + ' ' + data['New_FLAVORSEG']

pivot_table2 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,
    columns=['CC_taste'],
    values=['qty'],  
    aggfunc='sum'           
)

pivot_table2

qty  \
CC_taste                                                             CC Fresh   
YYYYMM id                                                 gender age            
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대      NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대      NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대      NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대      NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대      NaN   
...                                                                       ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대      NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대      NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대      NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대      1.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대      3.0   

                                                                                   \
CC_taste                                                             CC New Taste   
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대         35.0   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대         11.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          2.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대          NaN   

                                                                                 \
CC_taste                                                             CC Regular   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대        1.0   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대        NaN   
...                                                                         ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대        2.0   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대       31.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대        NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대        NaN   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대        NaN   

                                                                                \
CC_taste                                                             HnB Fresh   
YYYYMM id                                                 gender age             
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대       3.0   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대       NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대       NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대       NaN   
       0265

In [149]:

result =data[data['productSubFamilyCode'].isin(['AIIM', 'FIIT', 'HEETS', 'MIIX', 'NEO', 'NEOSTICKS', 'TEREA'])]

data['HnB_taste'] = result['productSubFamilyCode'] + ' ' + data['New_FLAVORSEG']

pivot_table3 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['HnB_taste'],
    values=['qty'],  # 피벗할 값
    aggfunc='sum'            # 집계 함수 (여기서는 평균)
)

pivot_table3

qty  \
HnB_taste                                                            AIIM Fresh   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대        NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대        NaN   
...                                                                         ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대        NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대        NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대        NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대        NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대        NaN   

                                                                                     \
HnB_taste                                                            AIIM New Taste   
YYYYMM id                                                 gender age                  
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대            NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대            NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대            NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대            NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대            NaN   
...                                                                             ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대            NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대            NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대            NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대            NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대            NaN   

                                                                                   \
HnB_taste                                                            AIIM Regular   
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대          NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대          NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          NaN   

                                                                                 \
HnB_taste                                                            FIIT Fresh   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
     

In [154]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS'][['engname']]
filtered_data['engname'].unique()

array(['TEREA BLUE', 'TEREA SUMMER WAVE', 'TEREA SUN PEARL',
       'TEREA PURPLE WAVE', 'TEREA TEAK', 'TEREA RUSSET', 'TEREA SILVER',
       'TEREA OASIS PEARL', 'TEREA BLACK PURPLE', 'TEREA YUGEN',
       'TEREA GREEN', 'TEREA AMBER', 'TEREA BLACK YELLOW',
       'HEETS AMBER LABEL', 'HEETS PURPLE LABEL', 'TEREA BLACK GREEN',
       'HEETS BLACK PURPLE SELECTION', 'HEETS YUGEN', 'TEREA GREEN ZING',
       'HEETS SILVER LABEL', 'HEETS GREEN LABEL', 'HEETS BLUE LABEL',
       'HEETS SATIN WAVE', 'HEETS BLACK GREEN SELECTION',
       'HEETS SUMMER BREEZE'], dtype=object)

In [158]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS']

pivot_table4 = filtered_data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['engname'],
    values=['qty'],  # 피벗할 값
    aggfunc='sum'            # 집계 함수 (여기서는 평균)
)

pivot_table4

qty  \
engname                                                              HEETS AMBER LABEL   
YYYYMM id                                                 gender age                     
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대               NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대               NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대               NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대               NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대               NaN   
...                                                                                ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대               NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대               NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대               NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대               NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대               NaN   

                                                                                                  \
engname                                                              HEETS BLACK GREEN SELECTION   
YYYYMM id                                                 gender age                               
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대                         NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대                         NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대                         NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대                         NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대                         NaN   
...                                                                                          ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대                         NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대                         NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대                         NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대                         NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대                         NaN   

                                                                                                   \
engname                                                              HEETS BLACK PURPLE SELECTION   
YYYYMM id                                                 gender age                                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대                          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대                          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대                          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대                          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대                          NaN   
...                                                                                           ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대                          NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대                          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대                          NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대                          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대                          NaN   

      

In [178]:
combined = pd.concat([pivot_table, pivot_table2, pivot_table3, pivot_table4], axis=1, keys=['YYYYMM', 'id', 'gender', 'age'] )

combined.sort_index(level='id')
combined.to_excel('finish.xlsx')